# Detección de Objetos usando YOLOv4



## Paquetes Necesarios y Configuración Previa

Si se desea trabajar con GPU en Google Colab (recomendado), se debe activar la opción de GPU en "Entorno de ejecución"/"Cambiar tipo de entorno de ejecución"



In [ ]:
# Clonar el repositorio de Darknet que contiene YOLO v4
!git clone https://github.com/AlexeyAB/darknet

In [ ]:
# Modificar el makefile para tener GPU y utilizar OPENCV
%cd /content/darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile

In [ ]:
# Compilar darknet usando make: compila "darknet"
!make

In [ ]:
# Pesos pre-entrenados de YOLOv4 (80 clases de COCO)
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.weights

In [ ]:
# Función para mostrar imagen en google colab usando matplotlib
import cv2
import matplotlib.pyplot as plt

def mostrar_imagen(path):
  #%matplotlib inline
  I = cv2.imread(path)
  h, w = I.shape[:2]
  I2 = cv2.resize(I,(3*w, 3*h), interpolation = cv2.INTER_CUBIC)
  fig = plt.gcf(); fig.set_size_inches(18, 10); plt.axis("off")
  plt.imshow(cv2.cvtColor(I2, cv2.COLOR_BGR2RGB))
  plt.show()

## Detección de Objetos

Se puede correr usando:
```bash
!./darknet   detector   test   <archivo .data>   <archivo config>   <archivo pesos>   <imagen>   -dont_show
```
La imagen detectada se almacena por defecto como 'predictions.jpg'

In [ ]:
# Utilizar "darknet" con algunas imágenes de prueba
!./darknet   detector   test   cfg/coco.data   cfg/yolov4.cfg   yolov4.weights   data/person.jpg   -dont_show

In [ ]:
# Mostrar la imagen en colab
mostrar_imagen('predictions.jpg')

In [ ]:
# Se puede probar con imágenes subidas a la máquina virtual de colab
!./darknet   detector   test   cfg/coco.data   cfg/yolov4.cfg   yolov4.weights ../imagen_prueba.jpg
mostrar_imagen('predictions.jpg')

In [ ]:
# Montar Google Drive para leer imágenes almacenadas allí
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# Creación de un enlace simbólico (por facilidad)
!ln   -s   /content/gdrive/My\ Drive/   /mydrive

In [ ]:
# Realizar detecciones usando las imágenes de Google Drive
%cd /content/darknet

!./darknet   detector   test   cfg/coco.data   cfg/yolov4.cfg   yolov4.weights   /mydrive/images/imagen_prueba.jpg   -dont_show

mostrar_imagen('predictions.jpg')

In [ ]:
# Para descargar se puede realizar manualmente, o copiando los archivos a google drive (que ya está montado)
!cp predictions.jpg /mydrive/images/resultado1.jpg

In [ ]:
# Detección en Video
!./darknet   detector   demo   cfg/coco.data   cfg/yolov4.cfg   yolov4.weights   -dont_show   /mydrive/images/video_prueba.mp4   -i   0   -out_filename   resultados.avi

In [ ]:
# Copiar el video a Google Drive
!cp  resultados.avi  /mydrive/images/resultados.avi

## Algunos parámetros de configuración

In [ ]:
# Threshold (-thresh)
# Solo detecta objetos cuya probabilidad sea mayor al nivel indicado

!./darknet   detector   test   cfg/coco.data   cfg/yolov4.cfg   yolov4.weights   data/dog.jpg   -thresh 0.5
mostrar_imagen('predictions.jpg')

In [ ]:
# Para obtener información sobre los "bounding boxes" se utiliza la opción -ext_output

!./darknet   detector   test   cfg/coco.data   cfg/yolov4.cfg   yolov4.weights   data/person.jpg   -ext_output   -dont_show
#mostrar_imagen('predictions.jpg')

In [ ]:
# Se puede almacenar los resultados de la detección en formato json usando la opción -out
!./darknet   detector   test   cfg/coco.data   cfg/yolov4.cfg   yolov4.weights   data/person.jpg   -ext_output   -dont_show  -out   resultado.json

In [ ]:
import json

with open("resultado.json") as f:
  datos = f.read()
res = json.loads(datos)

# Ejemplo: recuperar las coordenadas del primer objeto
res[0]['objects'][0]['relative_coordinates']